# Code for retrieving Spotify playlists

NOTE: The spotify API requires refreshing every 60 minutes, so if code does not work, you might need to rerun the entire notebook

In [1]:
import requests
import pandas as pd
import json

### Enter your client_id and client_secret here:
- Get this from the spotify API
- Reference for the code found below: Steven Morse (2020) https://stmorse.github.io/journal/spotify-api.html

In [2]:
## Enter your own Client_Id and Client_secret ##
CLIENT_ID = ''
CLIENT_SECRET = ''

In [3]:
AUTH_URL = 'https://accounts.spotify.com/api/token'
BASE_URL = 'https://api.spotify.com/v1/'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [4]:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# From here on I created the actual functions

This function goes through a playlist and retrieves:
- The artist
- The song name
- The song ID (this is needed for the next function)
- Song duration (in ms)
- Popularity (might be skewed towards the more recent songs though)
    - "According to Spotify, “popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are."" (Philip Peker, 2021) https://towardsdatascience.com/predicting-popularity-on-spotify-when-data-needs-culture-more-than-culture-needs-data-2ed3661f75f1

In [5]:
def playlist_songs(ID):
    r_playlist = requests.get(BASE_URL + 'playlists/' + ID, headers=headers)

    #looks at the specific songs within the playlist
    r2 = r_playlist.json()
    r3 = r2['tracks']
    r3.keys()
    r4 = r3['items']

    #puts results in dictionary
    data = []
    for r in r4:
        song_name = r['track']['name']
        song_url = r['track']['external_urls']['spotify']
        song_artist = r['track']['album']['artists'][0]['name']
        song_duration = r['track']['duration_ms']
        song_popularity = r['track']['popularity']
        # only gives the last part after the '/' in the url, giving the song ID
        song_id  = song_url.split('/')[-1]
        #song_id is the key because two songs can have the same name
        data.append({
            'artist': song_artist,
            'song_name': song_name,
            'song_id': song_id,
            'song_duration': song_duration,
            'song_popularity': song_popularity
        })
    return data

The following function uses the playlist_songs function, and adds the audiofeatures to them
As a result we get:
- name, song and id
- audio features like
    - danceability
    - speechiness
    - tempo
    - etc.


In [6]:
def add_audiofeatures(playlist_id):
    playlist = playlist_songs(playlist_id)
    for song in playlist:
        track_id = song['song_id']
        trysong = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)
        song.update(trysong.json())
    return playlist

In [7]:
euro2021 = add_audiofeatures("37i9dQZF1DWVCKO3xAlT1Q?si=236c30d415484365")
euro2020 = add_audiofeatures("0IyJJbmF3vp5nX6j4FFeDO?si=fc50f88c6eb54747")
euro2019 = add_audiofeatures("3ZdQUt8Tmtt7oOU8UM2koe?si=85c3bf9659eb4b4c")
euro2018 = add_audiofeatures("5sxwk5T34E2l2Ng02lipHS?si=600b20aa157f49da")
euro2017 = add_audiofeatures("6wECFlAQT9OTMCrdgGHPnz?si=219ac671992b4115")
euro2016 = add_audiofeatures("6ld9FVLMoZPxKezXf94AOK?si=8a1020be2e654345")
euro2015 = add_audiofeatures("0eW7WDU93NWFjDnKmqRelb?si=0e47c59fe0ba46e2")
euro2014 = add_audiofeatures("04U4zcFoZ8WqIDbhjnoxIv?si=cdf43bb693e74ae9")
euro2013 = add_audiofeatures("09OKlgunTlKElpxU6RHMQr?si=b28e0fc35d77413f")
euro2012 = add_audiofeatures("5tjy5qbWxaFFFTseRDptC7?si=8b010011750f4ab8")
euro2011 = add_audiofeatures("4HPdL5jwsxKnh7RTAwq9hF?si=59fc5fbb0bbe446c")
euro2010 = add_audiofeatures("3Vf3E8WavFCRLGF3a97nR9?si=f2114a3f8c6940d2")


# Quick look at what the dataframe will look like

In [8]:
pd.DataFrame(euro2021)

,artist,song_name,song_id,song_duration,song_popularity,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Måneskin,ZITTI E BUONI,1lWWoec2z1j88GRblI5anV,192654,72,0.620,0.944,4,-3.082,0,...,0.7330,0.5920,103.024,audio_features,1lWWoec2z1j88GRblI5anV,spotify:track:1lWWoec2z1j88GRblI5anV,https://api.spotify.com/v1/tracks/1lWWoec2z1j8...,https://api.spotify.com/v1/audio-analysis/1lWW...,192655,4
1,Destiny,Je me casse - Eurovision Official Entry,0NbGUXzlqYkSKlGiduwo84,178500,24,0.767,0.871,8,-4.494,0,...,0.2770,0.7900,111.973,audio_features,0NbGUXzlqYkSKlGiduwo84,spotify:track:0NbGUXzlqYkSKlGiduwo84,https://api.spotify.com/v1/tracks/0NbGUXzlqYkS...,https://api.spotify.com/v1/audio-analysis/0NbG...,178500,4
2,Elena Tsagrinou,El Diablo,4TAttqXwjj56xZQVKvlX0K,181546,44,0.660,0.664,9,-7.780,0,...,0.8320,0.6210,113.994,audio_features,4TAttqXwjj56xZQVKvlX0K,spotify:track:4TAttqXwjj56xZQVKvlX0K,https://api.spotify.com/v1/tracks/4TAttqXwjj56...,https://api.spotify.com/v1/audio-analysis/4TAt...,181547,4
3,Barbara Pravi,Voilà,1uAOCTevGnyKIDbgZdOCnE,176815,68,0.447,0.337,2,-8.665,0,...,0.2980,0.6030,135.379,audio_features,1uAOCTevGnyKIDbgZdOCnE,spotify:track:1uAOCTevGnyKIDbgZdOCnE,https://api.spotify.com/v1/tracks/1uAOCTevGnyK...,https://api.spotify.com/v1/audio-analysis/1uAO...,176816,4
4,THE ROOP,Discoteque,7dMOzsTZOUtOF7W5kLN0gf,180521,61,0.826,0.799,5,-6.120,1,...,0.1250,0.6280,115.004,audio_features,7dMOzsTZOUtOF7W5kLN0gf,spotify:track:7dMOzsTZOUtOF7W5kLN0gf,https://api.spotify.com/v1/tracks/7dMOzsTZOUtO...,https://api.spotify.com/v1/audio-analysis/7dMO...,180522,4
5,Tusse,Voices,5zxZ7M4RgWZUHlMEwac8vt,184000,64,0.503,0.759,10,-5.222,0,...,0.1300,0.3960,90.015,audio_features,5zxZ7M4RgWZUHlMEwac8vt,spotify:track:5zxZ7M4RgWZUHlMEwac8vt,https://api.spotify.com/v1/tracks/5zxZ7M4RgWZU...,https://api.spotify.com/v1/audio-analysis/5zxZ...,184000,4
6,Daði Freyr,10 Years,4O0LQKVT6hGmVGNwizmydg,166000,66,0.798,0.621,2,-6.916,0,...,0.0926,0.4430,123.038,audio_features,4O0LQKVT6hGmVGNwizmydg,spotify:track:4O0LQKVT6hGmVGNwizmydg,https://api.spotify.com/v1/tracks/4O0LQKVT6hGm...,https://api.spotify.com/v1/audio-analysis/4O0L...,166000,4
7,Senhit,Adrenalina,5sckZhskSmeF8TFJ05oSCB,179786,55,0.665,0.886,10,-5.773,0,...,0.3870,0.5290,104.084,audio_features,5sckZhskSmeF8TFJ05oSCB,spotify:track:5sckZhskSmeF8TFJ05oSCB,https://api.spotify.com/v1/tracks/5sckZhskSmeF...,https://api.spotify.com/v1/audio-analysis/5sck...,179787,4
8,Eden Alene,Set Me Free,76TRCnbYfo9GU5DPsZ1uI9,179439,58,0.752,0.664,1,-7.392,0,...,0.1020,0.4680,107.020,audio_features,76TRCnbYfo9GU5DPsZ1uI9,spotify:track:76TRCnbYfo9GU5DPsZ1uI9,https://api.spotify.com/v1/tracks/76TRCnbYfo9G...,https://api.spotify.com/v1/audio-analysis/76TR...,179439,4
9,Gjon's Tears,Tout l'univers,3MvY2XbGYFi6p2dgmdcOYG,183706,64,0.534,0.365,9,-9.187,0,...,0.1580,0.2370,123.466,audio_features,3MvY2XbGYFi6p2dgmdcOYG,spotify:track:3MvY2XbGYFi6p2dgmdcOYG,https://api.spotify.com/v1/tracks/3MvY2XbGYFi6...,https://api.spotify.com/v1/audio-analysis/3MvY...,183707,4


# This creates the json files that can be used for analysis

In [9]:
with open("euro2021.json", "w") as outfile:
    outfile.write(json.dumps(euro2021))
with open("euro2020.json", "w") as outfile:
    outfile.write(json.dumps(euro2020))
with open("euro2019.json", "w") as outfile:
    outfile.write(json.dumps(euro2019))
with open("euro2018.json", "w") as outfile:
    outfile.write(json.dumps(euro2018))
with open("euro2017.json", "w") as outfile:
    outfile.write(json.dumps(euro2017))
with open("euro2016.json", "w") as outfile:
    outfile.write(json.dumps(euro2016))
with open("euro2015.json", "w") as outfile:
    outfile.write(json.dumps(euro2015))
with open("euro2014.json", "w") as outfile:
    outfile.write(json.dumps(euro2014))
with open("euro2013.json", "w") as outfile:
    outfile.write(json.dumps(euro2013))
with open("euro2012.json", "w") as outfile:
    outfile.write(json.dumps(euro2012))
with open("euro2011.json", "w") as outfile:
    outfile.write(json.dumps(euro2011))
with open("euro2010.json", "w") as outfile:
    outfile.write(json.dumps(euro2010))